# Tuning Notebook: dfm
Dummy-Daten, kleine Splits, Stage A (3 Blöcke) + kurze Stage B. Exporte unter `outputs/`.

In [1]:

import os, sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent   # wenn Notebook in repo/notebooks liegt
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import GlobalConfig, DEFAULT_CORR_SPEC, EWMA_CORR_SPEC, outputs_for_model
from src.tuning import run_stageA, run_stageB
import numpy as np, pandas as pd


In [2]:
from src.models.dfm import ForecastModel

In [3]:

# Outputs sicherstellen
outputs_for_model("dfm")


{'stageA': PosixPath('/Users/jonasschernich/Documents/Masterarbeit/Code/outputs/stageA/dfm'),
 'stageB': PosixPath('/Users/jonasschernich/Documents/Masterarbeit/Code/outputs/stageB/dfm'),
 'logs': PosixPath('/Users/jonasschernich/Documents/Masterarbeit/Code/outputs/logs/dfm.log')}

In [10]:

from src.io_timesplits import load_target, load_ifo_features
import pandas as pd

# 1) Laden
from src.io_timesplits import load_target, load_ifo_features
y = load_target()
X = load_ifo_features()
idx = y.index.intersection(X.index)
y, X = y.loc[idx], X.loc[idx]

# 2) TSFresh/Chronos (optional)
import pandas as pd, src.features as F
tsfresh_df = pd.read_parquet(PROJECT_ROOT/"data/processed/tsfresh_slim.parquet").set_index("date")
chronos_df = pd.read_parquet(PROJECT_ROOT/"data/processed/chronos_1step.parquet").set_index("date")
tsfresh_df.index = pd.to_datetime(tsfresh_df.index)
chronos_df.index = pd.to_datetime(chronos_df.index)
tsfresh_df = tsfresh_df.reindex(y.index)
chronos_df = chronos_df.reindex(y.index)
F.tsfresh_block  = lambda y_s, I_t, W=12: tsfresh_df.loc[[y_s.index[I_t-1]]]
F.chronos_block  = lambda y_s, I_t, W=12: chronos_df.loc[[y_s.index[I_t-1]]]

# 3) Config
from src.config import GlobalConfig, DEFAULT_CORR_SPEC
cfg = GlobalConfig()
cfg.corr_spec = dict(DEFAULT_CORR_SPEC)
cfg.nuisance_seasonal = "auto"
cfg.lag_candidates = tuple(range(1,13))
cfg.k1_topk = 600
cfg.redundancy_method = "greedy"; cfg.redundancy_param = 0.9
cfg.W0_A = 180; cfg.BLOCKS_A = [(181,200),(201,220),(221,240)]
cfg.W0_B = 240

# 4) Run (DFM als Beispiel)
from src.tuning import run_stageA, run_stageB
from src.models.dfm import ForecastModel
shortlist = run_stageA("dfm", lambda hp: ForecastModel(hp), [{"alpha":0.0},{"alpha":0.1}], X, y, cfg)
run_stageB("dfm", lambda hp: ForecastModel(hp), shortlist, X, y, cfg)



In [13]:

# --- Stage A ---
shortlist = run_stageA(
    model_name="dfm",
    model_ctor=lambda hp: ForecastModel(hp),
    model_grid=model_grid,
    X=X, y=y, cfg=cfg
)
shortlist


[Stage A][Block 1] train_end=60, OOS=61-70 | configs=2
  - Config 1/2: {'alpha': 0.0}
    · Month 1/10 processed | running RMSE=2.1310
    · Month 2/10 processed | running RMSE=1.5366
    · Month 3/10 processed | running RMSE=1.8537
    · Month 4/10 processed | running RMSE=1.8815
    · Month 5/10 processed | running RMSE=1.7522
    · Month 6/10 processed | running RMSE=1.6234
    · Month 7/10 processed | running RMSE=1.6218
    · Month 8/10 processed | running RMSE=1.7785
    · Month 9/10 processed | running RMSE=1.6785
    · Month 10/10 processed | running RMSE=1.7412
  - Config 2/2: {'alpha': 0.1}
    · Month 1/10 processed | running RMSE=2.4391
    · Month 2/10 processed | running RMSE=1.7321
    · Month 3/10 processed | running RMSE=1.9372
    · Month 4/10 processed | running RMSE=2.0161
    · Month 5/10 processed | running RMSE=1.8801
    · Month 6/10 processed | running RMSE=1.7414
    · Month 7/10 processed | running RMSE=1.7249
    · Month 8/10 processed | running RMSE=1.8680


[{'alpha': 0.0}]

In [14]:

# --- Stage B (kurz) ---
run_stageB(
    model_name="dfm",
    model_ctor=lambda hp: ForecastModel(hp),
    shortlist=shortlist,
    X=X, y=y, cfg=cfg,
    max_months=6
)
print("Stage B done → outputs/stageB/dfm/")


[Stage B] Month origin t=100 | evaluating 1 configs | active=1
[Stage B] Month origin t=101 | evaluating 1 configs | active=1
[Stage B] Month origin t=102 | evaluating 1 configs | active=1
[Stage B] Month origin t=103 | evaluating 1 configs | active=1
[Stage B] Month origin t=104 | evaluating 1 configs | active=1
[Stage B] Month origin t=105 | evaluating 1 configs | active=1
Stage B done → outputs/stageB/dfm/
